In [90]:
# Reading PDF files and basic NLP
import PyPDF2
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Fuzzy string match
from fuzzywuzzy import fuzz

# To generate word clouds
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline

# Read unicode data from the extra_stop_words file
import unicodedata
import re
import os

# Frequency counting and collections
import numpy as np
import collections

# To navigate through all files in a directory
from os import listdir
from os.path import isfile, join

## 1. Define functions for the text extraction, preprocessing and analysis

In [91]:
# Adapted from https://gist.github.com/boniattirodrigo/67429ada53b7337d2e79
def remove_special_characters(word):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', word)
    plain_word = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # return the word with only numbers, letters and spaces
    return re.sub('[^a-zA-Z0-9 \\\]', '', plain_word)

In [92]:
# Adapted from https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f
# Updated the keywords removal 
def extract_keywords(path_pdf,
                     pdf_file,
                     path_output_raw_text,
                     stop_words_language,#='portuguese', 
                     path_extra_stop_words,#='/Users/hmg/Dropbox/veve e heitor/Projeto_tese',
                     file_extra_stop_words): #='extra_stop_words.txt'):
    
    # Check if the raw text is not already availabe in the path_output_raw_text
    text = ''
    try:
        candidate_raw = open(os.path.join(path_output_raw_text, pdf_file.replace('.pdf', '.txt')), 'r')
        text = candidate_raw.read()
#         print('successfully read the raw text! ')
    except IOError:
#         print('gotta scan the pdf...')
        # open allows you to read the file
        pdfFileObj = open(os.path.join(path_pdf, pdf_file), 'rb')
        # The pdfReader variable is a readable object that will be parsed
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        # discerning the number of pages will allow us to parse through all the pages
        num_pages = pdfReader.numPages
        # Just to show info about creator and creation time. 
        # print(str(pdfReader.getDocumentInfo())
        count = 0

        # The while loop will read each page
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count += 1
            text += pageObj.extractText()
        # This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned
        # files.
        if text != "":
            text = text
        # If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
        else: # fileurl
            text = textract.process(os.path.join(path_pdf, pdf_file), method='tesseract', language='eng', encoding='utf8')

        file_raw_text = open(os.path.join(path_output_raw_text, pdf_file.replace('.pdf', '.txt')), 'w', encoding='utf8')
        file_raw_text.write(text)
    
    text = remove_special_characters(text)
    # The word_tokenize() function will break our text phrases into #individual words
    tokens = word_tokenize(text)
    punctuations = ['(', ')', ';', ':', '[', ']', ',', '%', '-', '.', '|', '']
    stop_words = stopwords.words(stop_words_language)
    extra_stop_words = open(os.path.join(path_extra_stop_words, file_extra_stop_words), 'r', encoding='utf8').read().split('\n')
    
    # We create a list comprehension which only returns a list of words #that are NOT IN stop_words and NOT IN
    # punctuations.
    keywords = [word.lower() for word in tokens 
                if 
                not word.lower() in stop_words and 
                not word.lower() in punctuations and 
                not word.lower() in extra_stop_words and 
                word.isalpha() and
                len(word) >= 2]
    
    return np.asarray(keywords)

In [93]:
def word_cloud_from_keywords_frequency(keywords_frequency, file_name, path='./wordclouds/', show_image=False):
    wordcloud = WordCloud(width = 512, height = 512, background_color='white')
    fig = plt.figure(figsize=(20,16),facecolor = 'white', edgecolor='blue')
    plt.imshow(wordcloud.generate_from_frequencies(keywords_frequency), interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    
    if show_image:
        plt.show()
    plt.savefig(path+file_name)
    plt.close(fig)

In [94]:
def calculate_statistics(keywords):
    total_words = 0
    keywords_dict = dict(collections.Counter(keywords).most_common())
    for k in keywords_dict:
        total_words += keywords_dict[k]

    stats = dict()
    for k in keywords_dict:
        stats[k] = {'count': keywords_dict[k], 'text_frequency': keywords_dict[k]/float(total_words)}    
#     stats['frequency'].most_common(10)
    
    return stats

In [95]:
def write_report(path, output_file, keyword_stats):
    output = open(path+output_file, 'w', encoding='utf8')
    header = u'word,count,text_frequency\n'
    output.write(header)
    for k in keyword_stats:
        line = u'%s,%d,%.5f\n' % (k, keyword_stats[k]['count'], keyword_stats[k]['text_frequency'])
        output.write(line)
    output.close()

## 2. Count occurrences of any word

* All PDFs in a given path are processed.
* This function generates 'raw reports', counting the occurrence and frequency of every word of the document. 
* A word is defined as a sequence of characters delimited by empty spaces, i.e. ' '. 
* **TODO:** This version of the code still doesn't handle pdfs that are in PT and EN.

In [96]:
def process_PDFs(path='/Users/hmg/Desktop/data/PDF_relatorios_sustentabilidade/', 
                     raw_text_path='./raw_texts/EN/',
                     output_path='./reports/EN/', 
                     wordclouds_path='./wordclouds/EN/',
                     stop_words_language='english',
                     path_extra_stop_words='/Users/hmg/Dropbox/veve e heitor/Final_Dissertacao/',
                     file_extra_stop_words='extra_stop_words_EN.txt',
                     wordclouds=False, 
                     start_index=0):
    # We assume there are only pdfs in this directory
    PDFs = [f for f in listdir(path) if isfile(join(path, f))]
    PDFs = list(filter(lambda pdf: pdf.find('.pdf') != -1, PDFs))
        
    counter = start_index
    for pdf in PDFs[counter:]:
        
        print('Processing PDFs ({0}) {1} of {2} ({3})'.format(stop_words_language, counter+1, len(PDFs), pdf))
        keywords = extract_keywords(path, pdf, 
                                    path_output_raw_text=raw_text_path, 
                                    stop_words_language=stop_words_language,
                                    path_extra_stop_words=path_extra_stop_words,
                                    file_extra_stop_words=file_extra_stop_words)
        keywords_statistics = calculate_statistics(keywords)
        write_report(output_path, pdf.replace('.pdf', '.csv'), keywords_statistics)
        counter = counter + 1
        if wordclouds:
            word_cloud_from_keywords_frequency(collections.Counter(keywords), 
                                               pdf.replace('.pdf', '.png'), 
                                               path=wordclouds_path,
                                               show_image=False)

In [99]:
# ENglish PDFs processing
process_PDFs(path='/Users/hmg/Desktop/Data/pdfs_EN/', 
             raw_text_path='./raw_texts/EN/',
             output_path='./reports/EN/', 
             wordclouds_path='./wordclouds/EN/',
             stop_words_language='english',
             wordclouds=True,
            start_index = 0)

Processing PDFs (english) 1 of 209 (CCRO_1_2015_EN.pdf)
Processing PDFs (english) 2 of 209 (WAL_25_Non_2013_EN.pdf)
Processing PDFs (english) 3 of 209 (FIBR_7_2012_EN.pdf)


Processing PDFs (english) 4 of 209 (BUN_8_Non_2014_EN.pdf)
Processing PDFs (english) 5 of 209 (EMBR_19_2014_EN.pdf)
Processing PDFs (english) 6 of 209 (BRFS_8_2011_EN.pdf)
Processing PDFs (english) 7 of 209 (ITEC_23_2012_EN.pdf)
Processing PDFs (english) 8 of 209 (QGEP_10_2015_EN.pdf)
Processing PDFs (english) 9 of 209 (JBSS_8_2014_EN.pdf)
Processing PDFs (english) 10 of 209 (CBA_3_Non_2018_EN.pdf)
Processing PDFs (english) 11 of 209 (PSSA_14_2014_EN.pdf)
Processing PDFs (english) 12 of 209 (V_M_3_Non_2013_EN.pdf)
Processing PDFs (english) 13 of 209 (USI_10_Non_2014_EN.pdf)
Processing PDFs (english) 14 of 209 (BRSR_14_2012_EN.pdf)
Processing PDFs (english) 15 of 209 (NATU_15_2013_EN.pdf)
Processing PDFs (english) 16 of 209 (CMIG_2_2013_EN.pdf)
Processing PDFs (english) 17 of 209 (CAR_24_Non_2016_EN.pdf)
Processing PDFs (english) 18 of 209 (VER_7_Non_2011_EN.pdf)
Processing PDFs (english) 19 of 209 (ARMT_3_2015_EN.pdf)
Processing PDFs (english) 20 of 209 (POR_5_Non_2015_EN.pdf)
Processi

Processing PDFs (english) 144 of 209 (VIM_1_Non_2013_EN.pdf)
Processing PDFs (english) 145 of 209 (TIET S_2_2016_EN.pdf)
Processing PDFs (english) 146 of 209 (OXIT_16_2015_EN.pdf)
Processing PDFs (english) 147 of 209 (ODE_1_Non_2014.pdf)
Processing PDFs (english) 148 of 209 (VER_7_Non_2012_EN.pdf)
Processing PDFs (english) 149 of 209 (‎BRKM_16_2017_EN.pdf)
Processing PDFs (english) 150 of 209 (BBAS_14_2013_EN.pdf)
Processing PDFs (english) 151 of 209 (INV_12_Non_2013_EN.pdf)
Processing PDFs (english) 152 of 209 (DOW_16_Non_2014_EN.pdf)
Processing PDFs (english) 153 of 209 (HSB_14_Non_2011_EN.pdf)
Processing PDFs (english) 154 of 209 (ECOR_12_2017_EN.pdf)
Processing PDFs (english) 155 of 209 (CCRO_1_2012_EN.pdf)
Processing PDFs (english) 156 of 209 (VALE_26_2017_EN.pdf)
Processing PDFs (english) 157 of 209 (COP_24_Non_2012_EN.pdf)
Processing PDFs (english) 158 of 209 (PETR_10_2014_EN.pdf)
Processing PDFs (english) 159 of 209 (APX_13_Non_2016_EN.pdf)
Processing PDFs (english) 160 of 209 

In [102]:
# PorTuguese PDFs processing
for i in range(4,5): # Hardcoded 5 because I know there are only 4 directories... 
    print('Processing pdfs_PT{0}'.format(i))
    process_PDFs(path='/Users/hmg/Desktop/Data/pdfs_PT{0}/'.format(i), 
             raw_text_path='./raw_texts/PT/',
             output_path='./reports/PT/', 
             wordclouds_path='./wordclouds/PT/',
             stop_words_language='portuguese',
             file_extra_stop_words='extra_stop_words_PT.txt',
             wordclouds=True,
             start_index=0)

Processing pdfs_PT4
Processing PDFs (portuguese) 1 of 209 (UNI_9_Non_2018.pdf)
Processing PDFs (portuguese) 2 of 209 (UNIMED_12_2014.pdf)


Processing PDFs (portuguese) 3 of 209 (QUI_16_Non_2014.pdf)
Processing PDFs (portuguese) 4 of 209 (MRVE_21_2014.pdf)
Processing PDFs (portuguese) 5 of 209 (MET_5_Non_2017.pdf)
Processing PDFs (portuguese) 6 of 209 (MET_5_Non_2016.pdf)
Processing PDFs (portuguese) 7 of 209 (UNI_11_Non_2017.pdf)
Processing PDFs (portuguese) 8 of 209 (MRVE_21_2015.pdf)
Processing PDFs (portuguese) 9 of 209 (OUR_28_Non_2018.pdf)
Processing PDFs (portuguese) 10 of 209 (MPLU_12_2015.pdf)
Processing PDFs (portuguese) 11 of 209 (UNIMED_12_2015.pdf)
Processing PDFs (portuguese) 12 of 209 (QUI_16_Non_2015.pdf)
Processing PDFs (portuguese) 13 of 209 (QGEP_10_2016.pdf)
Processing PDFs (portuguese) 14 of 209 (MAR_17_Non_2014.pdf)
Processing PDFs (portuguese) 15 of 209 (SIORG_3_2013.pdf)
Processing PDFs (portuguese) 16 of 209 (QUI_16_Non_2017.pdf)
Processing PDFs (portuguese) 17 of 209 (UNIMED_12_2017.pdf)
Processing PDFs (portuguese) 18 of 209 (LREN_17_2017.pdf)
Processing PDFs (portuguese) 19 of 209 (MPLU_12_2017.

Processing PDFs (portuguese) 80 of 209 (Radiante_4_2016.pdf)
Processing PDFs (portuguese) 81 of 209 (MUL_22_Non_2013.pdf)
Processing PDFs (portuguese) 82 of 209 (USI_24_Non_2013.pdf)
Processing PDFs (portuguese) 83 of 209 (NEOE_10_2017.pdf)
Processing PDFs (portuguese) 84 of 209 (MMMC_1_2013.pdf)
Processing PDFs (portuguese) 85 of 209 (PLA_12_Non_2017.pdf)
Processing PDFs (portuguese) 86 of 209 (USA_15_Non_2015.pdf)
Processing PDFs (portuguese) 87 of 209 (PLA_12_Non_2015.pdf)
Processing PDFs (portuguese) 88 of 209 (ROC_28_Non_2016.pdf)
Processing PDFs (portuguese) 89 of 209 (USI_24_Non_2011.pdf)
Processing PDFs (portuguese) 90 of 209 (SIR_9_Non_2018.pdf)
Processing PDFs (portuguese) 91 of 209 (Malwee_17_2016.pdf)
Processing PDFs (portuguese) 92 of 209 (Malwee_17_2017.pdf)
Processing PDFs (portuguese) 93 of 209 (Radiante_4_2015.pdf)
Processing PDFs (portuguese) 94 of 209 (NEOE_10_2014.pdf)
Processing PDFs (portuguese) 95 of 209 (ROC_28_Non_2017.pdf)
Processing PDFs (portuguese) 96 of 20

## 3. Count occurrences of predefined phrases/words

**IN-DEVELOPMENT:** Count occurrences of predefined text using 'fuzzy string match', added ```word_match_count``` lambda object. 

In [70]:
def count_total_words(text, delim=' '): 
    return len(remove_special_characters(text).split(delim))

In [72]:
def count_normal(text, word, confidence = 0, phrases = False):
    clean_text = remove_special_characters(text)
    clean_word = remove_special_characters(word)
    if phrases:
        return clean_text.count(clean_word)
    else:
        counter = 0
        for p in clean_text.split(' '):
            if(p == clean_word):
                counter = counter + 1
        return counter

def count_fuzzy(text, word, confidence = 95, phrases=False):
    counter = 0
    if len(word) == 0:
        return 0
    # Phrases
    if phrases:
        # Sliding window strategy: create a text_word from text with length word.len 
        #  and by moving 1 character at a time
        #     If match, then skip the next word.len, just to avoid double counting!
        i = 0
        ## DEBUG
#         print('count_fuzzy - word = {0} and, len(text) = {1}, confidence = {2}'.format(word, len(text), confidence))
        print('PALAVRA = {0} - CONFIANCA = {1}'.format(word, confidence))
    
        while i < len(text):
            text_word_last_idx = i+len(word)
            match_confidence = 0
            if text_word_last_idx <= len(text):
                match_confidence = fuzz.ratio(text[i:text_word_last_idx], word)
#                 print('token_set_ratio({0},{1})={2}'.format(text[i:text_word_last_idx], word, match_confidence))
                if match_confidence > confidence:
                    counter += 1
                    print('{0} =~ {1} ({2})'.format(text[i:text_word_last_idx], word, match_confidence))
                    i += len(word)
                else:
                    i += 1
            else:
                break
            ## DEBUG - only about 100 characters
            if(i % int(len(text)/10) == 0):
                print('{} of {} = {:.2f}%'.format(i, len(text), i/len(text)*100))
    else:
        text_vec = text.split()
        for w in text_vec:
            match_confidence = fuzz.ratio(w, word)
            if match_confidence > confidence:
                counter += 1
    return counter

In [ ]:
def filter_special_keywords(filter_list_path='/Users/hmg/Dropbox/veve e heitor/Projeto_tese/', 
                            filter_list_file='lista_palavras_EN.txt', 
                            raw_text_path='./raw_texts/EN/',
                            output_path='./reports_special/EN/',
                            output_file='output_EN.csv',
                            word_match_count=count_normal,
                            confidence=95,
                            phrases=False):
    print('Starting filter_special_keywords')
    filter_words = ''
    try:
        filter_words = open(os.path.join(filter_list_path, filter_list_file), 'r', encoding='utf8').read().split('\n')
        print('successfully read the filter words at {0} named {1}'.format(filter_list_path, filter_list_file))
    except IOError:
        print('failed to read the filter words at {0} named {1}'.format(filter_list_path, filter_list_file))
    
    # assuming there are only the raw texts in the directory
    text_files = [f for f in listdir(raw_text_path) if isfile(join(raw_text_path, f))]
    text_files = list(filter(lambda text_file: text_file.find('.txt') != -1, text_files))
    
    stats_per_file = {}
    for text_file in text_files:
        try:
            text = open(os.path.join(raw_text_path, text_file), 'r').read()
        except IOError:
            print('Failed to open file at {0} named {1}'.format(raw_text_path, text_file))
#         print('Text file: {0}'.format(text_file))
        
        stats = {}
        for filter_word in filter_words:
            stats[filter_word] = word_match_count(text.lower(), filter_word.lower(), confidence, phrases)
            # text.lower().count(filter_word.lower())
#             line = u'%s,%d,%.5f\n' % (filter_word, counter, -1)

        stats_per_file[text_file.replace('.txt','')] = stats
        
    output = open(output_path+output_file, 'w', encoding='utf8')   
    header = u'word,' + u','.join(text_files)+u'\n'
    output.write(header)
    
    
    for filter_word in filter_words:
        output.write(filter_word + ',')
        for text_file in text_files:
#             line = u'%s,%d,%.5f\n' % (filter_word, counter, -1)
            output.write(str(stats_per_file[text_file.replace('.txt','')][filter_word]) + ',')
        output.write('\n')
    output.close()
    print('Finishing filter_special_keywords')

## Sandbox

In [ ]:
# print(len(PDFs))
# print(len(PDFs_PT))
# print(len(PDFs_EN))

In [ ]:
text = u'administrativa da joao Elekeiroz. GRI 2.3  joao| 2.4As vendas para o mercado interno 88 corresponderam a 88% da Receita Líquida da Elekeiroz em 2011. As exportações para\nmais de 30 países responderam pelos restantes 12%. Os produtos \norgânicos foram vendidos nos mercados interno e externo, enquanto os inorgânicos tiveram como destino o mercado interno. MISSÃO GRI 4.8  GRI 2.2 | 2.7 | 2.8 Buscar a melhoria contínua dos produtos e serviços oferecidos aos seus clientes internos e \nexternos; destacar-se em '
print('count = {0}'.format(count_total_words(text)))

count_normal(text, 'joão', phrases=False)

In [16]:
# keywords = extract_keywords('/Users/hmg/Downloads/', '3M_RS2014.pdf', '/Users/hmg/Downloads/')
# keywords_frequency = collections.Counter(keywords)
# 
# keywords_frequency.most_common(10)

In [15]:
# keywords = extract_keywords('/Users/hmg/Desktop/data/PDF_relatorios_sustentabilidade/FIBR_7_2012_EN.pdf', 
#                             stop_words_language='english')
# keywords_frequency = collections.Counter(keywords)

# word_cloud_from_keywords_frequency(keywords_frequency, file_name='hey.png', plot=False)

# keywords_frequency.most_common(10)

In [ ]:
# calculate_statistics(keywords)

In [46]:
text = 'test text with test words in the text test words'

testCount = count_normal(text, 'test', phrases = False)
phraseCount = count_normal(text, 'test words', phrases = True)
print('testCount = {0} and phraseCount = {1}'.format(testCount, phraseCount))

testCount = 3 and phraseCount = 2


In [ ]:
# filter_special_keywords(filter_list_path='/Users/hmg/Dropbox/veve e heitor/Projeto_tese/', 
#                             filter_list_file='lista_palavras_EN.txt', 
#                             raw_text_path='./raw_texts/EN/',
#                             output_path='./reports_special/EN/')

# filter_special_keywords(filter_list_path='/Users/hmg/Dropbox/veve e heitor/Projeto_tese/', 
filter_special_keywords(filter_list_path='./', 
                            filter_list_file='lista_frases_v3_dev.txt', 
                            raw_text_path='./raw_texts/PT_dev/',
                            output_path='./reports_special/PT/',
                            output_file='output_PT_dev_fuzzy_phrases95.csv',
                            word_match_count=count_fuzzy,
                            confidence=95,
                            phrases=True
                            )

In [18]:
# Tests with fuzzy string match
# fuzz.ratio(str.lower(u'Casarão'), str.lower(u'Caserão'))
# fuzz.token_set_ratio(str.lower(u'Casarão'), str.lower(u'Casârão'))

# normal_count = count_normal("mama mia this is a mama and there is no mia in this mama", "mãma")
# fuzzy_count = count_fuzzy("mama mia this is a mama and there is no mia in this mama", "mãma", 80)

# print('normal = {0}, fuzzy = {1}'.format(normal_count, fuzzy_count))

86

In [92]:
fuzz.ratio(str.lower(u'vacao da biodiversidade'), str.lower(u''))
# fuzz.token_set_ratio(str.lower(u'EN-13'), str.lower(u'EN-'))

92